In [34]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm

warnings.filterwarnings(action='ignore')
# warnings.filterwarnings(action='default')

In [35]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../data/'

In [36]:
train_df = pd.read_csv(data_path+'train.csv')
test_df = pd.read_csv(data_path+'test.csv')
submission_df = pd.read_csv(data_path+'sample_submission.csv')
target_df = pd.read_csv(data_path+'revealed_targets.csv')

In [37]:
# null 값 처리
train_df['far_price'] = train_df['far_price'].fillna(0)
train_df['near_price'] = train_df['near_price'].fillna(1)
train_df.isnull().sum()

stock_id                     0
date_id                      0
seconds_in_bucket            0
imbalance_size             220
imbalance_buy_sell_flag      0
reference_price            220
matched_size               220
far_price                    0
near_price                   0
bid_price                  220
bid_size                     0
ask_price                  220
ask_size                     0
wap                        220
target                      88
time_id                      0
row_id                       0
dtype: int64

In [38]:
cols_group_by = ['date_id', 'seconds_in_bucket']
cols_fill_nan = [
    'imbalance_size', 'reference_price', 'matched_size', 'wap',
    'bid_price', 'bid_size', 'ask_price', 'ask_size',
    'stock_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag']
train_grouped_median = train_df.groupby(cols_group_by)[cols_fill_nan].transform('median')
train_df[cols_fill_nan] = train_df[cols_fill_nan].fillna(train_grouped_median)
train_df.loc[train_df.isnull().any(axis=1)] # target 88

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,1381981.10,0,0.999950,9723622.86,0.0,1.0,0.999688,0.0,1.000242,0.0,1.000000,NaN,1925,35_0_131
369700,131,35,10,1371886.54,0,1.000252,9961197.49,0.0,1.0,0.999969,0.0,1.000485,0.0,1.000223,NaN,1926,35_10_131
369892,131,35,20,1331838.54,0,1.000122,9999133.11,0.0,1.0,0.999883,0.0,1.000328,0.0,1.000149,NaN,1927,35_20_131
370084,131,35,30,1350584.58,0,0.999910,10133596.07,0.0,1.0,0.999757,0.0,1.000186,0.0,0.999971,NaN,1928,35_30_131
370276,131,35,40,1327284.70,0,0.999926,10133596.07,0.0,1.0,0.999758,0.0,1.000203,0.0,0.999984,NaN,1929,35_40_131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225338,158,388,510,339056.84,0,1.001074,19640140.62,0.0,1.0,1.000867,0.0,1.001250,0.0,1.001027,NaN,21391,388_510_158
4225538,158,388,520,437746.70,0,1.001183,19767348.90,0.0,1.0,1.001007,0.0,1.001290,0.0,1.001163,NaN,21392,388_520_158
4225738,158,388,530,483535.46,0,1.000994,19800447.29,0.0,1.0,1.000905,0.0,1.001187,0.0,1.001032,NaN,21393,388_530_158
4225938,158,388,540,453894.62,0,1.000989,20020720.89,0.0,1.0,1.000824,0.0,1.001024,0.0,1.000911,NaN,21394,388_540_158


In [39]:
# (stock_id=131, date_id=35), (stock_id=158, date_id=388) target이 없으므로 제거
train_df = train_df.drop(index=train_df.loc[((train_df['stock_id']==131) & (train_df['date_id']==35))  | ((train_df['stock_id']==158) & (train_df['date_id']==388))].index)
train_df[train_df.isnull().any(axis=1)]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
3555081,101,328,0,3683316.08,0,0.999681,7405651.01,0.0,1.0,0.999647,0.0,1.000356,0.0,1.0,NaN,18040,328_0_101
4764999,19,438,0,2201071.62,-1,1.000146,6841177.82,0.0,1.0,0.999749,0.0,1.000277,0.0,1.0,NaN,24090,438_0_19


In [40]:
# stock_id=101, date_id=328, seconds_in_bucket=0의 target값을 stock_id=101, date_id=328, seconds_in_bucket=1의 target값으로 세팅
train_df.loc[train_df['row_id']=='328_0_101', 'target'] = train_df.loc[(train_df['stock_id']==101) & (train_df['time_id']==18041), 'target'].values
# stock_id=19, date_id=438, seconds_in_bucket=0의 target값을 stock_id=19, date_id=438, seconds_in_bucket=1의 target값으로 세팅
train_df.loc[train_df['row_id']=='438_0_19', 'target'] = train_df.loc[(train_df['stock_id']==19) & (train_df['time_id']==24091), 'target'].values
train_df.isnull().sum()

stock_id                   0
date_id                    0
seconds_in_bucket          0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
far_price                  0
near_price                 0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
row_id                     0
dtype: int64

In [41]:
from tqdm.auto import tqdm

X_all = None
y_all = None
selected_features = [
    'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price',
    'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size','wap'
]
for stock_id in tqdm(train_df['stock_id'].unique()):
    stock_data = train_df.loc[train_df['stock_id']==stock_id, selected_features].values
    y_data = train_df.loc[train_df['stock_id']==stock_id, 'target'].to_numpy()
    stock_data = stock_data.reshape(-1, 55, len(selected_features)).transpose(0,2,1)
    y_data = y_data.reshape(-1,55,1).transpose(0,2,1).squeeze() # (95234, 1, 55) =squeeze=> (95234, 55)
    if X_all is None:
        X_all = stock_data
        y_all = y_data
    else:
        X_all = np.concatenate((X_all, stock_data), axis=0)
        y_all = np.concatenate((y_all, y_data), axis=0)
X_all.shape, y_all.shape

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
valid_dataset = TensorDataset(torch.from_numpy(X_valid).float(), torch.from_numpy(y_valid).float())

In [ ]:
batch_size=2048
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
import torch.nn as nn

class Conv1dModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            # (N, Cin, Lin) => (N, Cout, Lout)
            # (N, 12, 55) => (N, 32, 52)
            nn.Conv1d(
                in_channels=12,
                out_channels=32,
                kernel_size=4,
                stride=1, padding=0, dilation=1,
                groups=1, bias=True, padding_mode='zeros'
            ),
            nn.LeakyReLU(),
            # (N, 32, 52) => (N, 32, 26)
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(0.1),
        )

        self.layer2 = nn.Sequential(
            # (N, 32, 26) => (N, 64, 24)
            nn.Conv1d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1, padding=0, dilation=1,
                groups=1, bias=True, padding_mode='zeros'
            ),
            nn.LeakyReLU(),
            # (N, 64, 24) => (N, 64, 12)
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(0.2),
        )

        self.fc = nn.Sequential(
            nn.Linear(64 * 12, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = torch.flatten(x, start_dim=1, end_dim=-1) # (batch_size, 64, 12) -> (batch_size, 64*12)
        x = self.fc(x)
        return x

In [ ]:
model = Conv1dModel().to(device)
model

In [ ]:
# x, y = next(iter(train_loader))
# x.shape, type(x)

In [ ]:
# model(x)

In [ ]:
# show params
for p in model.parameters():
    print(p.shape)

In [ ]:
import torch.optim as optim
learning_rate=0.0001
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.L1Loss()

In [ ]:
# Scheduler
from torch.optim import lr_scheduler

lr_scheduler = lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer=optimizer,
    T_0=20,
    T_mult=1,
    eta_min=1e-6
)
lr_scheduler

In [ ]:
epochs = 100

train_epoch_loss = []
valid_epoch_loss = []
for epoch in range(epochs):

    train_iter_loss = []
    bar = tqdm(enumerate(train_loader), total = len(train_loader), desc='Train Loop')
    model.train()
    for idx, (stocks, movements) in bar:
        # with torch.autocast(device_type='cuda', dtype=torch.float64):
        stocks = stocks.to(device)
        movements = movements.to(device)
        outputs = model(stocks)
        # print(outputs.shape, movements.shape)
        loss = criterion(outputs, movements)
        train_iter_loss.append(loss.item())
        optimizer.zero_grad() # 기울기 초기화
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        bar.set_postfix(
            Epoch = epoch,
            Current_loss = loss.item(),
            Train_loss = sum(train_iter_loss)/(idx+1),
            LR = optimizer.param_groups[0]['lr'],
        )
    print(f'Epoch [{epoch+1}/{epochs}] - Train loss : {sum(train_iter_loss)/len(train_loader):.4f}')
    train_epoch_loss.append(sum(train_iter_loss)/len(train_loader))

    model.eval()
    valid_iter_loss = []
    bar = tqdm(enumerate(valid_loader), total = len(valid_loader), desc='Valid Loop')
    for idx, (stocks, movements) in bar:
        # with torch.autocast(device_type='cuda', dtype=torch.float16):
        with torch.no_grad():
            stocks = stocks.to(device)
            movements = movements.to(device)
            outputs = model(stocks)
            loss = criterion(outputs, movements)
            valid_iter_loss.append(loss.item())
            bar.set_postfix(
              Epoch = epoch,
              Current_loss = loss.item(),
              Valid_loss = sum(valid_iter_loss)/(idx+1),
              LR = optimizer.param_groups[0]['lr'],
          )
    print(f'Epoch [{epoch+1}/{epochs}] - Valid loss : {sum(valid_iter_loss)/len(valid_loader):.4f}')
    valid_epoch_loss.append(sum(valid_iter_loss)/len(valid_loader))

In [ ]:
sns.lineplot(train_epoch_loss, label='train')
sns.lineplot(valid_epoch_loss, label='valid')